In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import tensorflow as tf
from googletrans import Translator
from urllib.parse import quote, unquote
import numpy as np
import click
import ast
import os

In [7]:

class ScrappingRecipe: 
    def __init__(self, start, stop, country, region, recipe_id, start_id, loop):
        self.start = start 
        self.stop = stop 
        self.country = country
        self.region = region
        self.recipe_id = recipe_id
        self.start_id = start_id
        self.loop  = loop

    def start_scrapping(self): 
        # start = id barang paling pertama
        # end = id barang paling terakhir
        data_nama_makanan = []
        # data_nama_bahan = []
        data_nama_gizi = []
        data_kandungan_gizi = []
        # data_kuantitas_bahan = []
        tata_cara = []

        # note : ganti isi dari code tergantung dari start dan end id 
        code = [x for x in range(self.start, self.stop)]
        for x in code: 
            # kemudian akan request sesuai dengan id yang dimulai dari start sampai end 
            # contoh https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/3170 ,maka id = 3170
            r = requests.get("https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/" + str(x))
            soup = BeautifulSoup(
                r.content, 
                'lxml'
            )

            # cari nama_makanan
            name_of_food = soup.find_all('h3')

            #  masukan semua data ke dalam list data_nama_makanan 
            for food_name in name_of_food: 
                temp = food_name.get_text()
                data_nama_makanan.append(temp)

            # mencari nama dari nutrisi, ada banyak cuma yang diperluin cuma 4
            nutrition_name = soup.find_all('strong')[:4]

            #  mencari data kandungan gizi
            nutrition_ingredient = soup.find_all('td', class_='roundOff')[:4]


            #  masukan data nama_nutrisi ke list data_nama_gizi
            for nutrition in nutrition_name: 
                data_nama_gizi.append(nutrition.get_text()) 
            
            #  masukan data kandungan gizi  ke list data_kandungan_gizi
            for nut_ingredient in nutrition_ingredient: 
                data_kandungan_gizi.append(nut_ingredient.get_text())
        
            
            get_instrucition = soup.find_all('div', id='steps')
            for instrucition in get_instrucition: 
                tata_cara.append(instrucition.get_text().strip('\n'))
                    
        # fixing data kandungan gizi final
        # karena list data_kandungan_gizi bentuknya seperti list 1D = [], maka akan sedikit  menyulitkan dalam menentukan apakah data_kandungan_gizi[i] adalah informasi  dari makananp[i] atau tidak jadi pecah menbjadi list dua dimensi
        data_kandungan_gizi_final = []
        left = 0
        right = left + 4
        #  left = start, right = stop 
        for i in range(len(data_kandungan_gizi)): 
            if left == len(data_kandungan_gizi): 
                break
            temp = data_kandungan_gizi[left:right]
            data_kandungan_gizi_final.append(temp)
            left = right
            right = right + 4
        # hasil akhir dari data_kandungan_gizi_final = [[len(4)], [len(4)]...........]

        # fixing data nutrisi  final
        data_nama_gizi_final = []
        left = 0
        right = left + 4
        for i in range(len(data_nama_gizi)): 
            if left == len(data_nama_gizi): 
                break
            temp = data_nama_gizi[left:right]
            data_nama_gizi_final.append(temp)
            left = right
            right = right + 4

        # sama seperti yang ada pada data_kandungan_gizi_final

        #  adding recipe id, category, dan region nama sebagai kolom baru
        id_final = []
        category = []
        region_name = []
        for i in range(len(data_nama_makanan)): 
            id_final.append([self.recipe_id + str(self.start_id)])
            category.append([self.country])
            region_name.append([self.region])
            self.start_id += 1

        #  final data 
        data_final = {
            'recipe_id' : id_final, 
            'nama_makanan': data_nama_makanan,
            'nama_gizi' : data_nama_gizi_final,
            'kandungan_nutrisi' : data_kandungan_gizi_final, 
            'kategori' : category , 
            'region' : region_name, 
            'instruksi' : tata_cara
        }

        return data_final
        #  convert ke excel
       

    def get_ingredients(self): 
        all = []
        code = [x for x in range(self.start, self.stop)]
        for i in code: 
            r = requests.get(f'https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/{str(i)}')
            soup = BeautifulSoup(
                        r.content, 
                        "html.parser")
            get_list = []
            table  = soup.find_all('a')
            for i in table: 
                if i.get_text() not in ['\n                          Genius Kitchen(food.com)\n\n                        \n', 'menu', 'How To Use', 'FAQs', 'CoSyLab', 'How To Use', 'FAQs', 'CoSyLab', 'Ingredients', 'Instructions','\xa0RecipeDB', 'menu', 'Home', 'How To Use', 'Receptors', 'FAQs', 'Contact Us', 'CoSyLab', 'Home', 'How To Use', 'Receptors', 'FAQs', 'Contact Us', 'CoSyLab', 'Recipe Overview', 'Ingredients', 'Processes - Utensils', 'Instructions', '\n                          All Recipes\n\n                        \n','Similar Recipes by Processes', 'Similar Recipes by Category Composition', '', 'Creative Commons Attribution-NonCommercial-ShareAlike 3.0 Unported License', 'Indraprastha Institute of Information Technology Delhi (IIIT-Delhi)', 'Dr. Ganesh Bagler', '', '','']:
                    get_list.append(str(i.get_text()))
            all.append(get_list)
        return all
             


    def get_ingredient_quantity(self):
        all  = []
        for i in range(self.start, self.stop): 
            temp = []
            r = requests.get(f'https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/{i}')
            soup = BeautifulSoup(
                        r.content, 
                        "html.parser")
            find_td_tag = soup.find_all('td')
            for i in find_td_tag :
                if i.get_text() in ['2 1/4', 'to taste', '1 1/2', '2/3', '3/4'] or  i.get_text() in [str(x) for x in range(1,100)] or i.get_text() in [f'1/{x}' for x in range(2, 100)] : 
                    temp.append(i.get_text())
            all.append(temp)
        return all

  

    def get_unit_state(self, data_ingredients): 
        units_final = []
        state_final = []
        iteration = 0
        for i in range(self.start, self.stop): 
            r = requests.get(f'https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/{i}')
            soup = BeautifulSoup(
                            r.content, 
                            "html.parser")
            name_of_bahan_all = soup.find_all('tbody')
            states =  []
           
            index = 0 
            units = []
            all_td_tags = []
            all_ingredients = data_ingredients
            for tbody in name_of_bahan_all:
                td_tags =  list(soup.find_all('td'))
                for i in td_tags:
                    all_td_tags.append(i.get_text())
                for j in all_td_tags:
                    
                    for k in all_ingredients[iteration]: 
                        if k in j : 
                            #  special case
                            if iteration == 0 and self.country == 'Indonesia' and j == 'cloves': 
                                continue
                            units.append(all_td_tags[index + 2])
                            states.append(all_td_tags[index + 3])
                    index += 1
                iteration += 1
            units_final.append(units)
            state_final.append(states)
     
        return units_final, state_final
         

         

    def get_image_link(self):
        all_image_link = []
        for i in range(self.start, self.stop):  
            r = requests.get(f'https://cosylab.iiitd.edu.in/recipedb/search_recipeInfo/{i}')
            soup = BeautifulSoup(
                            r.content, 
                            "html.parser")
            get_img  = str(soup.find_all('img', class_= 'responsive-img')).split(" ")
            all_image_link.append([get_img[3].replace('src="', '').replace('"', '')])
        return all_image_link

    # def translate_column(self, get_data: pd.DataFrame, attr: str): 
    #     translated = []
    #     translator = Translator()
    #     row = 0
    #     data = get_data
    #     for i in data[attr]: 
    #         for j in i: 
    #           translated.append(translator.translate(j, dest='id').text) 
    #         data[attr].iloc[row] = translated
    #         row += 1
    #         translated = []
    #     return data

    def to_excel(self):
        preview_data  = self.start_scrapping()
        ingredients = self.get_ingredients()
        ingredients_quantity = self.get_ingredient_quantity()
        units, state = self.get_unit_state(data_ingredients= self.get_ingredients())
        image = self.get_image_link()

        final_data = {
            'recipe_id' : preview_data['recipe_id'],
            'nama_makanan' : preview_data['nama_makanan'],
            'nama_gizi' : preview_data['nama_gizi'],
            'kandungan_nutrisi' : preview_data['kandungan_nutrisi'], 
            'kategori' : preview_data['kategori'],
            'wilayah' : preview_data['region'], 
            'nama_bahan' : ingredients,
            'kuantitas' : ingredients_quantity,
            'satuan' : units, 
            'cara_mengolah' : state,
            'instruksi' : preview_data['instruksi'],
            'gambar' : image

        }
        # print(pd.DataFrame(final_data))
        return pd.DataFrame(final_data).to_excel('./final/' + self.country + self.loop + '.xlsx')
       
      

class get_price_recipe:
    def __init__(self, attr, data, country):
        self.data = data
        self.attr = attr
        self.country = country
    def get_bahan_encode(self): 
        encode = []
        for i in self.data[self.attr]: 
            for j in ast.literal_eval(i):
                # print(j) 
                encode.append(quote(j))
        return encode
    def get_all_price(self):
        data_encoded = self.get_bahan_encode()
        prices = {}
        temp_prices = []
        temp_bahan = []
        for j in data_encoded: 
            r = requests.get(f'https://www.sayurbox.com/search?q={j}&search_referrer_page=home&section_source=search_search_bar&touch_point=home&input_type=enter_search_keyword')
            soup = BeautifulSoup(r.content,  "html.parser")
            get_price_web = soup.find_all('span', class_ = 'css-1jxf684 r-1ymgsq6 r-1yr78gv r-1ulplbh r-6dt33c')
            temp = []
            total  = 0
            for i in get_price_web: 
                if 'Rp' in str(i.get_text()):
                    temp.append(i.get_text().replace('\n', '').strip())
            for i in temp: 
                total += int(i.replace('Rp', '').replace('.', '')) 
            
            if len(temp) != 0:
                temp_bahan.append(unquote(j))
                temp_prices.append(int((total) / len(temp)) - 2000) 
                print(f'{unquote(j)} berhasil ditambahkan dan dikalkulasi dengan harga  : {int((total) / len(temp)) - 2000}')   
        prices['nama_bahan'] = temp_bahan
        prices['harga'] = temp_prices
        return pd.DataFrame(prices).to_excel('./final/harga_bahan_makan_' +  str(self.country) + '.xlsx')
    

class mix_all_loop: 
    def __init__(self,  file1, file2, country):
        self.file1 = file1
        self.file2 = file2
        self.country = country
    def merge(self):
        read_file_1 = pd.read_excel(self.file1)   
        read_file_2 = pd.read_excel(self.file2)
        pd.concat([read_file_1, read_file_2],ignore_index=True).to_excel('./final/final' + self.country + '.xlsx', columns=['recipe_id', 'nama_makanan', 'nama_gizi','kandungan_nutrisi', 'kategori',  'wilayah', 'nama_bahan', 'kuantitas', 'satuan',  'cara_mengolah','instruksi', 'gambar' ])
         



In [8]:



test_class1 = ScrappingRecipe(start=4003, stop=4454, country='India', region='Asia', recipe_id='IND', start_id=1, loop='1')
ingredients = test_class1.to_excel() 


#  code : 
# INDONESIA
# 1. Indonesia (1) = 3435 - 3467
# 2. Indonesia (2)  = 59670 - 59996

# INDIAN
# 1. Indian (1) = 
# 2. Indian (2)  = 


#  tutorial pakai (untuk scrapping resep makanan (12 features)): 
# 1. buat contoh object untuk class1 
#  contoh : test_class1 = ScrappingRecipe(start=59670, stop=59996, country='Indonesia', region='Asia', recipe_id='ID', start_id=34, loop='2')
#  keterangan parameter : start = start code resep,  end : end code resep, loop : ini tuh kalian scrapping yang keberapa kali untuk satu negara karena kan id bisa tiba" beda misal kek indonesia yang awalnya 3456 tiba" 59670 - 59996,  start_id  : karena id di url web bisa gak sama di baris tertentu maka set ulang start_id misal scrappinag pertama cuma dapet 33 berati start id untuk scrapping kedua start_id = 34
# 2. buat folder dengan nama 'final' ('./final/)
# 3. kemudian pangggil method to_excel()
# 4. ingat biasanya scrapping bagian ini dilakukan dua kali karena url id yang bisa berubah)
# 5. buat object untuk mix_all_loop class )
# contoh : test3 = mix_all_loop(file1='./final/Indonesia1.xlsx', file2='./final/Indonesia2.xlsx', country='Indonesia', file1 = file hasil scrapping pertama, file2 = hasil scrapping kedua
# 6. selesai 


# tutorial get_price_recipe
# 1. pastikan sudah melakukan tutorial yang ada diatas
# 2. terjenmahkan hasil dari tutorial diatas mwenggunakan translate -> https://translate.google.com/?hl=id&sl=en&tl=id&text=subscriptable&op=translate (klik bagian dokukmen setelah itu donwload file translatenya)
# 3. read file yang sudah didapatkan dari tutorial diatas : misal (get_data_bahan = pd.read_excel('./final./translated/finalIndonesia.xlsx'))
# 4. buat objek kelas get_price_recipe
# contoh : test_class2 = get_price_recipe(attr='nama_bahan', data=get_data_bahan[:10], country='Indonesia'), attr = nama_kolom (dan ini selalu bernilai = 'nama_bahan')
# 5. kemudian panggil metthod get_all_price()
# 6. selesai (note : scrapping harga ini agak lama dan mungkin aja rawan error time out, jadi saran sih jangan sekalian semua tapi satu persatu mungkin setiap 50 )




In [223]:
# ingredients

In [6]:
test3 = mix_all_loop(file1='./final/Thailand1.xlsx', file2='./final/Thailand2.xlsx', country='Thailand')
test3.merge()

In [22]:
get_data_bahan = pd.read_excel('./final./translated/finalIndonesia.xlsx')
# get_data_bahan['nama_bahan'].head()
test_class2 = get_price_recipe(attr='nama_bahan', data=get_data_bahan[:10], country='Indonesia')
count = 0
test_class2.get_all_price()
# get_data_bahan.iloc[:10]

minyak zaitun berhasil ditambahkan dan dikalkulasi dengan harga  : 42004
bawang merah berhasil ditambahkan dan dikalkulasi dengan harga  : 20775
cabai merah berhasil ditambahkan dan dikalkulasi dengan harga  : 25183
bawang putih berhasil ditambahkan dan dikalkulasi dengan harga  : 11808
terasi berhasil ditambahkan dan dikalkulasi dengan harga  : 4900
serai berhasil ditambahkan dan dikalkulasi dengan harga  : 6283
jahe berhasil ditambahkan dan dikalkulasi dengan harga  : 15100
kunyit berhasil ditambahkan dan dikalkulasi dengan harga  : 4312
santan berhasil ditambahkan dan dikalkulasi dengan harga  : 9518
gula palem berhasil ditambahkan dan dikalkulasi dengan harga  : 15750
kafir daun jeruk berhasil ditambahkan dan dikalkulasi dengan harga  : 17766
kayu manis berhasil ditambahkan dan dikalkulasi dengan harga  : 21091
cengkeh berhasil ditambahkan dan dikalkulasi dengan harga  : 48271
garam laut berhasil ditambahkan dan dikalkulasi dengan harga  : 28516
lada putih berhasil ditambahkan dan 